In [1]:
import cv2
import numpy as np
import math
import scipy.spatial.distance

In [2]:
# Фильтр Калмана для предсказания траектории

kf = cv2.KalmanFilter(4,2)
kf.measurementMatrix = np.array([[1,0,0,0],[0,1,0,0]], np.float32)
kf.transitionMatrix = np.array([[1,0,1,0],[0,1,0,1],[0,0,1,0], [0,0,0,1]], np.float32)

In [3]:
# Функция оценки позиции объекта

def predict(x, y):

    measured = np.array([[np.float32(x)], [np.float32(y)]])
    kf.correct(measured)
    predicted = kf.predict()
    x1, y1 = int(predicted[0]), int(predicted[1])
    
    return x1, y1

In [4]:
# Функция детектирования

def detect(frame, object_detector):
    
    mask = object_detector.apply(frame)
    _, mask = cv2.threshold(mask, 254, 255, cv2.THRESH_BINARY)
    
    # Распознование контуров
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    detections = []
    box = (0,0,0,0)
    for contour in contours:
        # Отчиста от мелких элементов
        area = cv2.contourArea(contour)
        if area > 1000:
#             cv2.drawContours(frame, [contour], -1, (0, 255, 0), 2)
            (x, y, w, h) = cv2.boundingRect(contour)
            box = (x, y, w, h)
            detections.append([x, y, w, h])
            
    return detections, mask, box

In [5]:
# Трекинг объектов

class DistTracker:
    def __init__(self):
        
        self.center_points = {}
        # Счётчик для новых объектов
        self.id_count = 1

    def update(self, objects_rect):
        # Список координат и id объектов
        objects_list = []

        for rect in objects_rect:
            x, y, w, h = rect
            # Центр объекта
            cx = (x + x + w) // 2
            cy = (y + y + h) // 2

            # Был ли объект ранее обнаружен
            same_object_detected = False
            for idx, pt in self.center_points.items():
                dist = scipy.spatial.distance.euclidean(cx - pt[0], cy - pt[1])

                if dist < 40:
                    self.center_points[idx] = (cx, cy)
#                     print(self.center_points)
                    objects_list.append([x, y, w, h, idx])
                    same_object_detected = True
                    break

            # При обнаружении нового объекта, необходимо присвоить ему новый ID
            if same_object_detected is False:
                self.center_points[self.id_count] = (cx, cy)
                objects_list.append([x, y, w, h, self.id_count])
                self.id_count += 1

        new_center_points = {}
        for obj_bb_id in objects_list:
            _, _, _, _, object_id = obj_bb_id
            center = self.center_points[object_id]
            new_center_points[object_id] = center
            
        self.center_points = new_center_points.copy()
        
        return objects_list

In [19]:
tracker = DistTracker()
cap = cv2.VideoCapture('balls_3.mp4')
object_detector = cv2.createBackgroundSubtractorMOG2(history=100, varThreshold=40)

In [20]:
while True:
    ret, frame = cap.read() 
    # Выход из цикла при окончании кадров видео
    if ret is False:
        break
        
    # 1.Детектирование
    detections, mask, box = detect(frame, object_detector)
    x,y,w,h = box
       
    # 2.Трекинг
    boxes_ids = tracker.update(detections)
    for box_id in boxes_ids:
        x, y, w, h, idd = box_id
        cv2.putText(frame, str(idd), (x, y - 15), cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 0), 2)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 3)
        
        # 3.Предсказание
        cx = (x + x + w) // 2
        cy = (y + y + h) // 2
        predicted = predict(cx, cy)
        cv2.circle(frame, (cx, cy), 10, (0, 0, 255), 3)
        cv2.circle(frame, (predicted[0], predicted[1]), 10, (255, 0, 0), -1)
    
    cv2.imshow("Frame", frame)
    cv2.imshow("Mask", mask)
    
    key = cv2.waitKey(80)
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()